---

# ☸️ 11. Docker with Kubernetes (K8s)

## A) Mental model — Container vs Pod

| Aspect  | Docker Container | K8s Pod                                       |
| ------- | ---------------- | --------------------------------------------- |
| Unit    | Single container | **Smallest deployable** (1+ containers)       |
| IP      | Per container    | **One IP shared** within Pod                  |
| Storage | Per container    | **Shared volumes** in Pod                     |
| Use     | Standalone       | Managed by **controllers** (Deployment/Job/…) |

> You deploy **Pods via Deployments**, not raw containers.

---

## B) Build → Push → Deploy (OCI image flow)

1. **Build**: `docker build -t yourrepo/myapp:v1 .`
2. **Push**: `docker push yourrepo/myapp:v1`
3. **Reference**:

```yaml
containers:
  - name: myapp
    image: yourrepo/myapp:v1
```

* Private registry → add `imagePullSecrets`.
* Prefer **immutable** tags/digests for prod.

---

## C) Core building blocks (when to use)

* **Deployment**: stateless apps, rolling updates.
* **StatefulSet**: stateful apps (stable IDs, ordered start).
* **DaemonSet**: one Pod per node (agents).
* **Job/CronJob**: one-off / scheduled work.
* **Service**: stable virtual IP (ClusterIP/NodePort/LB).
* **Ingress**: HTTP(S) routing to Services.
* **ConfigMap/Secret**: configuration & sensitive data.
* **PV/PVC**: persistent storage claims.
* **Namespace**: environment/tenant isolation.
* **HPA**: scale by CPU/RAM/custom metrics.

---

## D) Service & networking (cheat sheet)

* **Types**: ClusterIP (internal), NodePort, LoadBalancer, Headless (`clusterIP: None`).
* **DNS**: `svc.ns.svc.cluster.local` via CoreDNS.
* **Expose locally**: `kubectl port-forward svc/myapp 8080:80 -n app`.
* Ingress needs an **Ingress Controller** (e.g., NGINX).

---

## E) Storage (quick)

* Use **StorageClass** for dynamic PVCs.
* Access modes: **RWO** (single writer), **RWX** (multi).
* Mount with `volumeMounts` in Pod; never write to container image paths you’ll later mount over.

---

## F) Config & secrets

```yaml
envFrom:
  - configMapRef: { name: app-config }
  - secretRef:    { name: app-secret }
# or mount secret/config as files
```

* Keep secrets out of images; use **`stringData`** for easy authoring.

---

## G) Health & lifecycle

* **Probes**: readiness (gate traffic), liveness (restart), startup (slow boots).
* **initContainers**: pre-work before main container.
* **Graceful stop**: `terminationGracePeriodSeconds`.

---

## H) Resources, autoscaling & availability

```yaml
resources:
  requests: { cpu: "200m", memory: "256Mi" }
  limits:   { cpu: "500m", memory: "512Mi" }
```

* **HPA** (CPU example):
  `kubectl autoscale deploy myapp -n app --cpu-percent=70 --min=2 --max=5`
* **PDB** keeps N pods during maintenance:

```yaml
apiVersion: policy/v1
kind: PodDisruptionBudget
metadata: { name: myapp-pdb, namespace: app }
spec:
  minAvailable: 1
  selector: { matchLabels: { app: myapp } }
```

---

## I) Security (essentials)

* Run as **non-root**, drop caps:

```yaml
securityContext:
  runAsNonRoot: true
  allowPrivilegeEscalation: false
  capabilities: { drop: ["ALL"] }
```

* Use **ServiceAccount + RBAC**; restrict network via **NetworkPolicy**.
* Image policy: pinned tags/digests, **`IfNotPresent`** (dev) / **`Always`** (CI latest).

---

## J) Minimal, production-flavored stack

```yaml
apiVersion: v1
kind: Namespace
metadata: { name: app }
---
apiVersion: v1
kind: Secret
metadata: { name: regcred, namespace: app }
type: kubernetes.io/dockerconfigjson
data: { .dockerconfigjson: "<base64-json>" }
---
apiVersion: v1
kind: ConfigMap
metadata: { name: app-config, namespace: app }
data: { APP_ENV: "prod" }
---
apiVersion: v1
kind: Secret
metadata: { name: app-secret, namespace: app }
type: Opaque
stringData: { DB_PASSWORD: "s3cr3t" }
---
apiVersion: apps/v1
kind: Deployment
metadata: { name: myapp, namespace: app }
spec:
  replicas: 2
  selector: { matchLabels: { app: myapp } }
  template:
    metadata: { labels: { app: myapp } }
    spec:
      imagePullSecrets: [{ name: regcred }]
      securityContext:
        runAsNonRoot: true
        allowPrivilegeEscalation: false
      containers:
        - name: myapp
          image: yourrepo/myapp@sha256:<digest>
          ports: [{ containerPort: 8080 }]
          envFrom:
            - configMapRef: { name: app-config }
            - secretRef:    { name: app-secret }
          resources:
            requests: { cpu: "200m", memory: "256Mi" }
            limits:   { cpu: "500m", memory: "512Mi" }
          readinessProbe:
            httpGet: { path: /health, port: 8080 }
          livenessProbe:
            httpGet: { path: /health, port: 8080 }
---
apiVersion: v1
kind: Service
metadata: { name: myapp-svc, namespace: app }
spec:
  selector: { app: myapp }
  ports: [{ port: 80, targetPort: 8080 }]
  type: ClusterIP
```

---

## K) Local clusters (fast dev)

* **kind**, **minikube**, **Docker Desktop K8s**:

```bash
kind create cluster
kubectl config get-contexts && kubectl config use-context kind-kind
```

---

## L) `kubectl` quick ops (what they do)

```bash
kubectl apply -f k8s/                  # create/update
kubectl get pods,svc,deploy -n app     # list
kubectl describe pod <pod> -n app      # deep debug
kubectl logs -f deploy/myapp -n app    # stream logs
kubectl exec -it deploy/myapp -n app -- sh
kubectl port-forward svc/myapp-svc 8080:80 -n app
kubectl rollout status|undo deploy/myapp -n app
kubectl scale deploy/myapp -n app --replicas=3
```

---

## M) Compose → K8s mapping (quick)

| Compose      | K8s                            |
| ------------ | ------------------------------ |
| `services`   | **Deployment** + **Service**   |
| `volumes`    | **PV/PVC**                     |
| `depends_on` | **initContainers** / readiness |
| `networks`   | **Service + DNS**              |
| `.env`       | **ConfigMap/Secret**           |

> That’s everything you need—compact, ordered, and production-lean.
